In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [ ]:
def generate_sine_wave_data(num_samples=100, noise=0.1):
    """
    Generate sine wave data with noise.
    """
    # 选择一个随机的起始点
    start = np.random.uniform(0, 2 * np.pi)
    # 生成均匀分布的x值
    x = np.linspace(start, start + 2*np.pi, num_samples)
    y = np.sin(x) + np.random.normal(0, noise, num_samples)
    return x, y

In [3]:
# 生成随机噪声序列
def generate_random_noise(num_samples=100, noise=0.1):
    """
    Generate random noise data.
    """
    x = np.linspace(0, 2 * np.pi, num_samples)
    y = np.random.normal(0, noise, num_samples)
    return x, y

In [4]:
# 生成数据
def generate_sequences(n_samples_per_class, sequence_length):
    """
    Generate sequences of sine waves and random noise.
    """
    sequences_wave = [generate_sine_wave_data(sequence_length)[1] for _ in range(n_samples_per_class)]
    sequences_noise = [generate_random_noise(sequence_length)[1] for _ in range(n_samples_per_class)]
    sequences = np.array(sequences_wave + sequences_noise)
    labels = np.array([0] * n_samples_per_class + [1] * n_samples_per_class)
    return sequences, labels
    
# 数据处理
n_samples_per_class = 1000
sequence_length = 100
sequences, labels = generate_sequences(n_samples_per_class, sequence_length)

In [5]:
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2)

# 转换为PyTorch张量
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(2)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(2)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

In [6]:
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32)

In [7]:
# 构建RNN模型
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        output, _ = self.rnn(x)
        # output的形状是 (N, L, H)，其中N是批大小，L是序列长度，H是隐藏层大小    
        # 输出序列的最后1项时 (N, 1, H)
        last_output = output[:, -1]
        # 分类器将输出 (N, 1, num_classes)
        out = self.fc(last_output)
        # 最后的输出是(N, num_classes)
        return out.view(-1, self.num_classes)

In [8]:
def mini_batch_train(data_loader,model,optimizer,loss_fn):
    mini_batch_losses = []
    for x_batch, y_batch in data_loader:
        # 置为训练状态
        model.train()
        # Step 1 - 前向计算预测值
        yhat = model(x_batch)
        # Step 2 - 计算损失
        mini_batch_loss = loss_fn(yhat, y_batch)
        # Step 3 - 计算梯度
        mini_batch_loss.backward()
        # Step 4 - 参数更新
        optimizer.step()
        optimizer.zero_grad()

        mini_batch_losses.append(mini_batch_loss.item())
                  
    loss = np.mean(mini_batch_losses)
    return loss

In [9]:
def mini_batch_val(data_loader,model,loss_fn):
    mini_batch_losses = []
    for x_batch, y_batch in data_loader:
        # 置为验证状态
        model.eval()
        # Step 1 - 前向计算预测值
        yhat = model(x_batch)
        # Step 2 - 计算损失
        mini_batch_loss = loss_fn(yhat, y_batch)
        mini_batch_losses.append(mini_batch_loss.item())
                  
    loss = np.mean(mini_batch_losses)
    return loss

In [10]:
def train(model, train_loader, test_loader, loss_fn, optimizer, epochs):
    # 指定随机数种子，可以再现数据
    np.random.seed(23)
    torch.manual_seed(23)

    # 循环轮数计数
    total_epochs = 0

    losses = []  # 每轮训练的损失
    val_losses = [] #  每轮验证数据的损失

    for epoch in range(epochs):
        model.train()
        total_epochs += 1

        # 进入mini-batch的内循环
        loss = mini_batch_train(train_loader,model,optimizer,loss_fn)
        losses.append(loss)

        # 验证数据集（验证不需要计算梯度）
        with torch.no_grad():
            # 验证集的 mini_batch
            val_loss = mini_batch_val(test_loader,model,loss_fn)
            val_losses.append(val_loss)
    
    return losses, val_losses

In [11]:
# 初始化模型、损失函数和优化器
input_size = 1
hidden_size = 32
num_classes = 1
epochs = 1000
model = SimpleRNN(input_size, hidden_size, num_classes)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_loss, val_loss = train(model, train_loader, test_loader, loss_fn, optimizer, epochs)

In [14]:
with torch.inference_mode():
    outputs = model(X_test)
    predicted = torch.sigmoid(outputs).round()
    # 计算准确率
    accuracy = (predicted == y_test).float().mean()
    print(f'Accuracy: {accuracy.item() * 100:.2f}%')

Accuracy: 100.00%
